In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import praw
import time
import datetime
from datetime import datetime, timedelta

In [3]:
client = MongoClient('localhost', 27017)
db = client.db
headlines = db.headlines
threads = db.threads
analysis = db.analysis

In [4]:
df = pd.read_csv('companylist.csv')

In [5]:
secretfile = open('secret.txt')
reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secretfile.read(), user_agent='sentiment')

In [6]:
wantedTags = ['Daily Discussion', 'DD', 'Technicals', 'Fundamentals', 'Discussion', 'YOLO', 'Stocks']
subreddit = reddit.subreddit('wallstreetbets')
postlist = []

for post in subreddit.hot(limit = 1000):
    if len(postlist) < 100:
        postTime = post.created
        postDate = datetime.utcfromtimestamp(postTime)
    
        currentTime = datetime.utcnow()
    
        postDelta = currentTime - postDate
    
        postDelta = str(postDelta)
    
        if 'day' not in postDelta and post.link_flair_text in wantedTags:
            postlist.append(post)
        print(len(postlist))
    else:
        break

0
0
0
0
0
0
1
1
2
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
5
5
5
6
6
6
6
6
6
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
11
12
12
12
12
12
12
12
12
12
12
12
13
14
14
14
14
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
18
19
19
20
20
21
21
22
22
22
23
23
23
23
23
23
23
24
25
25
25
25
25
25
26
26
26
26
26
27
27
27
28
28
28
28
28
28
29
29
29
30
31
32
32
32
32
33
33
33
34
34
34
34
34
34
34
34
35
35
36
36
37
37
38
38
38
38
39
39
40
40
41
41
42
43
43
43
43
43
43
43
43
43
43
44
45
46
47
47
48
49
50
50
50
50
50
50
50
51
51
51
52
53
54
55
56
56
57
58
59
59
59
59
59
59
59
59
59
59
59
60
61
62
62
62
62
62
63
63
63
63
63
64
65
65
65
65
66
66
66
66
66
67
67
67
67
67
67
67
67
67
67
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
69
69
69
70
70
70
70
70
70
70
70
70
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71

In [7]:
len(postlist)

71